# **Install and Load Packages**

In [ ]:
# Install tensorflow-addons
! pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, AveragePooling2D, concatenate
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
import numpy as np
import pickle
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, ReLU, Concatenate, AveragePooling2D
# Import the unsupervised context normalization layer
import os
import sys
package_dir = os.getcwd()
root_dir = os.path.dirname(package_dir)
sys.path.append(root_dir)
from normalization.layers import UnsupervisedContextNormalization

# **Functions for standard normaization and save metrics and loss**

In [ ]:
# Compute Mean and Standard Deviation
def compute_mean_std(dataset):
    data_r = np.dstack([dataset[i][:, :, 0] for i in range(len(dataset))])
    data_g = np.dstack([dataset[i][:, :, 1] for i in range(len(dataset))])
    data_b = np.dstack([dataset[i][:, :, 2] for i in range(len(dataset))])
    mean = np.mean(data_r), np.mean(data_g), np.mean(data_b)
    std = np.std(data_r), np.std(data_g), np.std(data_b)
    return mean, std

In [ ]:
# Save list to binary file
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')

# Read list to memory
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

# **Define General DenseNet Architecture**

In [ ]:
def dense_block(x, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
    for _ in range(num_layers):
        y = BatchNormalization()(x)
        y = ReLU()(y)
        y = Conv2D(bn_size * growth_rate, kernel_size=1, strides=1, use_bias=False)(y)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        y = Conv2D(growth_rate, kernel_size=3, strides=1, padding='same', use_bias=False)(y)
        if drop_rate > 0:
            y = tf.keras.layers.Dropout(drop_rate)(y)
        x = Concatenate()([x, y])
        num_input_features += growth_rate
    return x, num_input_features

def transition(x, num_input_features, num_output_features):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(num_output_features, kernel_size=1, strides=1, use_bias=False)(x)
    x = AveragePooling2D(pool_size=2, strides=2)(x)
    return x, num_output_features // 2

def DenseNet(num_layers, growth_rate=12, num_classes=100):
    input_tensor = Input(shape=(32, 32, 3))
    input_norm = UnsupervisedContextNormalization(num_components=10, epsilon=1e-3, momentum=0.9)(input_tensor, training=True)
    x = Conv2D(2 * growth_rate, kernel_size=3, strides=1, padding='same', use_bias=False)(input_norm)
    num_features = 2 * growth_rate

    for i in range(3):
        x, num_features = dense_block(x, num_layers, num_features, 4, growth_rate, 0.2)
        if i < 2:
            x, num_features = transition(x, num_features, num_features // 2)

    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=x)
    return model


# **DenseNet-40**

In [ ]:
# Create DenseNet with 40 Conv layers
model_densenet40 = DenseNet(6)

In [ ]:
# Compile the models with Nesterov's accelerated gradient, weight decay, and momentum
num_classes = 100
batch_size = 64
epochs = 200
sgd = tf.keras.optimizers.SGD(0.1,momentum=0.9, nesterov=True,weight_decay=1e-4)
model_densenet40.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=[
            tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
            tf.keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tfa.metrics.F1Score(num_classes=num_classes, name="f1-score")

        ])

In [ ]:
# Define a learning rate schedule
def lr_schedule(epoch):
    if epoch > 100 and epoch < 150:
        return 0.01
    elif epoch > 150:
        return 0.001
    return 0.1

In [ ]:
# Load the CIFAR-100 dataset and perform data augmentation
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Preprocess CIFAR-100 dataset
mean, std = compute_mean_std(x_train)
x_train = x_train.astype('float32')
x_train = (x_train - mean) / std
x_test = x_test.astype('float32')
x_test = (x_test - mean) / std

In [ ]:
# Data Generator
train_size = 40000
x_val = x_train[train_size:]
y_val = y_train[train_size:]

x_train = x_train[:train_size]
y_train = y_train[:train_size]

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)
train_gen = datagen.flow(x=x_train, y=y_train, batch_size=batch_size)

In [ ]:
STEPS_PER_EPOCH = x_train.shape[0] // batch_size
SAVE_PERIOD = 50

In [ ]:
# Directory to  save checkpoints
checkpoint_dir = '.'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_weights_{epoch:03d}.h5'),
    save_weights_only=True,
    save_best_only=False,
    save_freq=SAVE_PERIOD*STEPS_PER_EPOCH # Sauvegarder tous les 50 époques
)

# Train the DenseNet-40 model
history = model_densenet40.fit(train_gen,
                     steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,
                     validation_data=(x_val, y_val), callbacks=[keras.callbacks.LearningRateScheduler(lr_schedule), checkpoint_callback])

In [ ]:
# Evaluate the model
loss, accuracy, top_5_accuracy, precision, recall, f1 = model_densenet40.evaluate(x_test, y_test)
print(f"Test loss : {loss}")
print(f"Test accuracy : {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy : {round(top_5_accuracy * 100, 2)}%")
print(f"Precision : {round(precision * 100, 2)}%")
print(f"Recall : {round(recall * 100, 2)}%")
print(f"F1-score : {f1}%")

In [ ]:
# Save the metrics and loss
write_list(history.history['accuracy'], 'accuracy')
write_list(history.history['val_accuracy'], 'val_accuracy')
write_list(history.history['loss'], 'loss')
write_list(history.history['val_loss'], 'val_loss')
write_list(history.history['precision'], 'precision')
write_list(history.history['val_precision'], 'val_precision')
write_list(history.history['recall'], 'recall')
write_list(history.history['val_recall'], 'val_recall')
write_list(history.history['val_f1-score'], 'val_f1')
write_list(history.history['f1-score'], 'f1')

# **DenseNet-100**

In [ ]:
# Create DenseNet-100
model_densenet100 = DenseNet(16)

In [ ]:
# Compile the models with Nesterov's accelerated gradient, weight decay, and momentum
num_classes = 100
batch_size = 64
epochs = 200
sgd = tf.keras.optimizers.SGD(0.1,momentum=0.9, nesterov=True,weight_decay=1e-4)
model_densenet100.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=[
            tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
            tf.keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tfa.metrics.F1Score(num_classes=num_classes, name="f1-score")

        ])

In [ ]:
# Define a learning rate schedule
def lr_schedule(epoch):
    if epoch > 100 and epoch < 150:
        return 0.01
    elif epoch >= 150:
        return 0.001
    return 0.1

In [ ]:
# Load the CIFAR-100 dataset and perform data augmentation
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Preprocess CIFAR-100 dataset
mean, std = compute_mean_std(x_train)
x_train = x_train.astype('float32')
x_train = (x_train - mean) / std
x_test = x_test.astype('float32')
x_test = (x_test - mean) / std

In [ ]:
# Data Generator
train_size = 40000
x_val = x_train[train_size:]
y_val = y_train[train_size:]

x_train = x_train[:train_size]
y_train = y_train[:train_size]

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)
train_gen = datagen.flow(x=x_train, y=y_train, batch_size=batch_size)

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    "model.ckpt",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)
# Train the DenseNet-100 model
history = model_densenet100.fit(train_gen,
                     steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,
                     validation_data=(x_val, y_val), callbacks=[keras.callbacks.LearningRateScheduler(lr_schedule), checkpoint_callback])


In [ ]:
# Evaluate model in the test set
loss, accuracy, top_5_accuracy, precision, recall, f1 = model_densenet100.evaluate(x_test, y_test)
print(f"Test loss : {loss}")
print(f"Test accuracy : {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy : {round(top_5_accuracy * 100, 2)}%")
print(f"Precision : {round(precision * 100, 2)}%")
print(f"Recall : {round(recall * 100, 2)}%")
print(f"F1-score : {f1}%")

In [ ]:
# Save metrics and loss
write_list(history.history['accuracy'], 'accuracy')
write_list(history.history['val_accuracy'], 'val_accuracy')
write_list(history.history['loss'], 'loss')
write_list(history.history['val_loss'], 'val_loss')
write_list(history.history['precision'], 'precision')
write_list(history.history['val_precision'], 'val_precision')
write_list(history.history['recall'], 'recall')
write_list(history.history['val_recall'], 'val_recall')
write_list(history.history['val_f1-score'], 'val_f1')
write_list(history.history['f1-score'], 'f1')